In [ ]:
import numpy as np

In [ ]:
# Environment and Policy Setup
# State space: S0 (Start), S_A (Target), S_B, S_C, S_D (Terminal)
# Rewards are obtained upon transitioning to the next state.
STATES = ['S0', 'S_A', 'S_B', 'S_C']
TERMINAL_STATE = 'S_D'

In [ ]:
TRANSITIONS = [
    ('S0', 'S_A', 0),
    ('S_A', 'S_B', 1),  # Visit 1 to S_A: Immediate Reward 1
    ('S_B', 'S_C', 0),
    ('S_C', 'S_A', 0),  # Loop back to S_A: Reward 0
    ('S_A', 'S_D', 10), # Visit 2 to S_A: Terminal Reward 10 (High reward for reaching the end)
]

In [ ]:
# Function to generate one episode 
def generate_episode():
    episode = []
    # State, Reward pair for the sequence
    episode.append(('S0', 0))
    episode.append(('S_A', 1)) # R1 (from S_A to S_B)
    episode.append(('S_B', 0)) # R2 (from S_B to S_C)
    episode.append(('S_C', 0)) # R3 (from S_C to S_A)
    episode.append(('S_A', 10))# R4 (from S_A to S_D)
    # The last state (S_D) doesn't yield a reward, so we stop here.
    return episode